In [0]:
!pip install praw

     |████████████████████████████████| 143kB 3.5MB/s 
     |████████████████████████████████| 204kB 8.9MB/s 


In [0]:
import requests
import requests.auth
import praw
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Setting up the reddit api to get details from various posts

In [0]:
client_auth = requests.auth.HTTPBasicAuth(@Client_id, @Client_secret)
post_data = {"grant_type": "password", "username": @username, "password": @password}
headers = {"User-Agent": "reddit_scraper"}
response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
response.json()

{'access_token': '284251727896-icg7SJe04MmHmMmMa5OpWlAIzo0',
 'expires_in': 3600,
 'scope': '*',
 'token_type': 'bearer'}

In [0]:
reddit = praw.Reddit(client_id=@Client_id, client_secret=@Client_secret, user_agent='reddit_scraper')

Getting the top 100 posts and top 10 comments from r/india habung the flairs mentioned below

In [0]:
subreddit = reddit.subreddit('india')
flairs = ["AskIndia", "Non-Political", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy","Sports", "Food", "AMA"]
posts = []
for i in flairs:
  red = subreddit.search(f"flair_name:{i}",limit=100)
  for post in red:
    temp = []
    temp.extend((post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created, i))
    post.comments.replace_more(limit=0)
    j=0
    for comment in post.comments.list():
      if j==10:
        temp.append(cmt)
        break
      if j == 0:
        cmt = str(comment.body)
      else:
        cmt = cmt + " " + str(comment.body)
      j += 1
    posts.append(temp)
print(posts)

In [0]:
postsdf = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created', 'flair', 'comments'])

In [0]:
postsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1100 non-null   object 
 1   score         1100 non-null   int64  
 2   id            1100 non-null   object 
 3   subreddit     1100 non-null   object 
 4   url           1100 non-null   object 
 5   num_comments  1100 non-null   int64  
 6   body          1100 non-null   object 
 7   created       1100 non-null   float64
 8   flair         1100 non-null   object 
 9   comments      846 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 86.1+ KB


Cleaning the text removing punctuationas and stop words

In [0]:
r1 = re.compile('[/(){}\[\]\|@,;]')
r2 = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = r1.sub(' ', text) # replace certain symbols by space in text
    text = r2.sub('', text) # delete symbols from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove STOPWORDS from text
    return text

In [0]:
postsdf['title'] = postsdf['title'].apply(str)
postsdf['body'] = postsdf['body'].apply(str)
postsdf['comments'] = postsdf['comments'].apply(str)

In [0]:
postsdf['title'] = postsdf['title'].apply(clean_text)
postsdf['body'] = postsdf['body'].apply(clean_text)
postsdf['comments'] = postsdf['comments'].apply(clean_text)

In [0]:
postsdf.to_csv("data.csv")

MODELS

---



In [0]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
import xgboost as xgb

In [0]:
df = pd.read_csv('/content/data.csv')
flairs = ["AskIndia", "Non-Political", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy","Sports", "Food", "AMA"]
df.fillna("",inplace=True)

In [0]:
Creating various features

In [0]:
f1 = df['title']
f2 = df['comments']
f3 = df['url']
f4 = df['body']
f5 = df['title'] +" "+ df['comments'] +" "+ df['url'] +" "+ df['body']
f6 = df['title'] +" "+ df['comments'] +" "+ df['url']
fl = df['flair']

In [0]:
Creating models to train on the features which were made

In [0]:
def runModel(data, fl):
  x_train, x_test, y_train, y_test = train_test_split(data, fl, test_size=0.2, random_state = 42)
  vect = CountVectorizer()
  tfidf = TfidfTransformer()
  models = [LogisticRegression(n_jobs=1, C=1e5, max_iter=4000),
            svm.SVC(), 
            MultinomialNB(), 
            RandomForestClassifier(n_estimators = 1000, random_state = 42), 
            xgb.XGBClassifier(), 
            MLPClassifier(hidden_layer_sizes=(30,30,30)), 
            SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)]
  pred = []
  acc = []
  for i in models:
    pipeline = Pipeline([('vect',vect), ('tfidf',tfidf), ('clf',i)])
    pipeline.fit(x_train, y_train)
    temp = pipeline.predict(x_test)
    pred.append(temp)
    acc.append(accuracy_score(temp, y_test))
  return pred, acc

In [0]:
Finding the best model

In [0]:
a1,b1 = runModel(f1,fl)
print(max(b1))

0.5409090909090909


In [0]:
a2,b2 = runModel(f2,fl)
print(max(b2))

0.5227272727272727


In [0]:
a3,b3 = runModel(f3,fl)
print(max(b3))

0.42727272727272725


In [0]:
a4,b4 = runModel(f4,fl)
print(max(b1))

0.5409090909090909


In [0]:
a5,b5 = runModel(f5,fl)
print(max(b5))

0.6909090909090909


Converting the best model into a pickle file to put on flask server

In [0]:
x_train, x_test, y_train, y_test = train_test_split(f5, fl, test_size=0.2, random_state = 42)
vect = CountVectorizer()
tfidf = TfidfTransformer()
pipeline = Pipeline([('vect',vect), ('tfidf',tfidf), ('clf',LogisticRegression(n_jobs=1, C=1e5, max_iter=4000))])
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=100000.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=

In [0]:
import pickle

In [0]:
pickle.dump(pipeline,open("model.pkl",'wb'))